In [1]:
import numpy as np
from pathlib import Path as pt

from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split

In [2]:
base_loc = pt('/Users/aravindhnivas/Documents/ML-properties/[PHYSICAL CONSTANTS OF ORGANIC COMPOUNDS]/tmp_C_processed_data/analysis_data/filtered/tmpC_topelements_processed_data/embedded_vectors')

embeddings = 'mol2vec_embeddings'
processed_vec_dir = base_loc / f'processed_{embeddings}'
X = np.load(processed_vec_dir / 'processed.X.npy', allow_pickle=True)
y = np.load(processed_vec_dir / 'processed.y.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape


((5980, 300), (5980,), (1496, 300), (1496,))

In [3]:
gbr_model = GradientBoostingRegressor()
xgb_model = XGBRegressor(verbosity=0)
cat_model = CatBoostRegressor(verbose=0)
lgb_model = LGBMRegressor(verbose=-1)

In [4]:
%%timeit
gbr_model.fit(X_train, y_train)

41.3 s ± 553 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
xgb_model.fit(X_train, y_train)

1.26 s ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
cat_model.fit(X_train, y_train)

8.15 s ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
lgb_model.fit(X_train, y_train)

902 ms ± 31.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
